In [ ]:
import pandas as pd

In [ ]:
interest_rate = pd.read_csv("./대출금리.csv" )
buy_index = pd.read_csv("./아파트매매가격지수.csv")
year_index = pd.read_csv("./아파트전세가격지수.csv")
seoul_gu = pd.read_csv("./서울시 구.csv")

# 1. 전처리

### 1. 주택금리 전처리

##### 1.1) 주택담보대출 (연리%)만 가져오기

In [ ]:
interest_rate2 = interest_rate[interest_rate['계정항목별'] == '주택담보대출 (연리%)'].transpose().reset_index(drop=False).iloc[1:]
interest_rate2.columns = ['날짜', '주택담보대출 (연리%)']

1.2) 날짜 형식 통일

In [ ]:
interest_rate2['날짜'] = pd.to_datetime(interest_rate2['날짜'])

In [ ]:
interest_rate3 = interest_rate2.set_index('날짜')

### 2. 아파트 매매가격지수 전처리

#### 2.1) 서울시만 가져오기

In [ ]:
gu_list = seoul_gu['구'].to_list()

In [ ]:
buy_index2 = buy_index.rename(columns={'구분':'날짜'}, inplace = 'TRUE')
buy_index3 = buy_index.set_index('날짜')[gu_list].iloc[1:]

#### 2.1) 월별 평균 지수로 변경

In [ ]:
buy_index3 = buy_index3.reset_index()
buy_index3['날짜'] = pd.to_datetime(buy_index3['날짜'])
buy_index3['month'] = buy_index3['날짜'].dt.month
buy_index3['year'] = buy_index3['날짜'].dt.year

In [ ]:
# 수치형으로 변경
forint = buy_index3.columns.difference(['year', 'month','날짜'])
buy_index3[forint] = buy_index3[forint].apply(pd.to_numeric, errors='coerce')

In [ ]:
#월별 평균지수
buy_index3 = buy_index3.groupby(['year','month']).mean().reset_index()
buy_index3['날짜'] = buy_index3['year'].astype(str) + '-' + buy_index3['month'].astype(str)
buy_index3['날짜'] = pd.to_datetime(buy_index3['날짜'], format='%Y-%m')
buy_index3 = buy_index3.drop(columns = ['month','year'], axis = 1)

<ipython-input-11-744469dd52d1>:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  buy_index3 = buy_index3.groupby(['year','month']).mean().reset_index()


In [ ]:
#2015년 1월1일 ~ 2023년 11월 1일까지만 가져오기
buy_index4 = buy_index3[(buy_index3['날짜'] >= '2015-01-01') & (buy_index3['날짜'] <= '2023-11-01')]

### 3. 아파트 전세가격지수 전처리

In [ ]:
year_index2 = year_index.rename(columns={'구분':'날짜'}, inplace = 'TRUE')
year_index3 = year_index.set_index('날짜')[gu_list].iloc[1:]
year_index3 = year_index3.reset_index()
year_index3['날짜'] = pd.to_datetime(year_index3['날짜'])
year_index3['month'] = year_index3['날짜'].dt.month
year_index3['year'] = year_index3['날짜'].dt.year

In [ ]:
# 수치형으로 변경
forint = year_index3.columns.difference(['year', 'month','날짜'])
year_index3[forint] = year_index3[forint].apply(pd.to_numeric, errors='coerce')

#월별 평균지수
year_index3 = year_index3.groupby(['year','month']).mean().reset_index()
year_index3['날짜'] = year_index3['year'].astype(str) + '-' + year_index3['month'].astype(str)
year_index3['날짜'] = pd.to_datetime(year_index3['날짜'], format='%Y-%m')
year_index3 = year_index3.drop(columns = ['month','year'], axis = 1)

<ipython-input-14-bf03bfd976b4>:6: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  year_index3 = year_index3.groupby(['year','month']).mean().reset_index()


In [ ]:
#2015년 1월1일 ~ 2023년 11월 1일까지만 가져오기
year_index4 = year_index3[(year_index3['날짜'] >= '2015-01-01') & (year_index3['날짜'] <= '2023-11-01')]

### 4. 임대료 데이터 만들기
전세/매매가격  * 주택담보대출금리 / 12

In [ ]:
buy_index5 = buy_index4.set_index('날짜')
year_index5 = year_index4.set_index('날짜')

In [ ]:
rent = year_index5 / buy_index5 #전세/매매가격

In [ ]:
columns = rent.columns
rent = rent.reset_index()
for i in columns  :
  rent[i] = rent[i] / interest_rate3.reset_index()['주택담보대출 (연리%)'] /12
rent = rent.set_index('날짜').apply(pd.to_numeric, errors='coerce')

In [ ]:
rent.to_csv("./rent.csv")
buy_index5.to_csv("./buy.csv")